In [1]:
import xnat
import os
import shutil

In [2]:
from data_io import filter_series

In [3]:
session = xnat.connect('https://bifrost.icr.ac.uk:8443/XNAT_anonymised/',user='tbarfoot',password='-----')

In [4]:
project = session.projects["MALIMAR_ALL"]
mrSession = project.experiments['20171204_125025_Avanto']
malimarSeries = filter_series(mrSession)

Scan <MrScanData fl3d_vibe_dixon_TRA_15deg 256_pocS_in (50)> does not contain a DICOM resource! oh


In [7]:
malimarSeries.xnat

{'inPhase': [<MrScanData T1 (59)>],
 'outPhase': [<MrScanData T1 OPP (60)>],
 'fat': [<MrScanData T1 FAT (62)>],
 'water': [<MrScanData T1 WAT (63)>],
 'b50': [<MrScanData 50 (23)>, <MrScanData B50 (42)>],
 'b600': [<MrScanData 600 (24)>, <MrScanData B600 (37)>],
 'b900': [<MrScanData 900 (25)>,
  <MrScanData 900.2 (26)>,
  <MrScanData B900 (32)>],
 'adc': [<MrScanData ADC (45)>],
 'diff': []}

In [9]:
for key in malimarSeries.xnat:
    print(key)

inPhase
outPhase
fat
water
b50
b600
b900
adc
diff


In [36]:
complete = (1,1,1,1,1,0,1,1,0)

In [37]:
a = []

In [38]:
for key, c in zip(malimarSeries.xnat,complete):
    a.append(len(malimarSeries.xnat[key])-c)

Complete if:

In [39]:
min(a) > -1 

True

Duplicates if:

In [40]:
max(a) > 0

True

actually no as having b600 would trigger this conditon as well. Write a seperate function that looks at whether len of each list in dictionary is greater than 1

In [41]:
len(malimarSeries.xnat)

9

In [45]:
keys = ['inPhase', 'outPhase', 'fat', 'water', 'b50', 'b600', 'b900', 'adc', 'diff']

In [48]:
import pandas as pd

In [ ]:
pd.D

In [50]:
dict.fromkeys(keys,[pd.DataFrame])

{'inPhase': [pandas.core.frame.DataFrame],
 'outPhase': [pandas.core.frame.DataFrame],
 'fat': [pandas.core.frame.DataFrame],
 'water': [pandas.core.frame.DataFrame],
 'b50': [pandas.core.frame.DataFrame],
 'b600': [pandas.core.frame.DataFrame],
 'b900': [pandas.core.frame.DataFrame],
 'adc': [pandas.core.frame.DataFrame],
 'diff': [pandas.core.frame.DataFrame]}

In [102]:
xnat_paths = {'dix': {'inPhase': [], 'outPhase': [], 'fat': [], 'water': []},
                           'dif': {'b50': [], 'b600': [], 'b900': [], 'adc': [], 'diff': []}}

In [110]:
xnat_paths['dix']['inPhase']

['in']

In [112]:
for group in xnat_paths:
    for series in xnat_paths[group]:
        for item in xnat_paths[group][series]:
            print(item)

in
out
fat
wat
5
6
9
a


In [115]:
xnat_paths = {'dix': {'inPhase': ['in'], 'outPhase': ['out'], 'fat': ['fat'], 'water': ['wat']},
              'dif': {'b50': ['a','b'], 'b600': ['c','d'], 'b900': ['e','f','g'], 'adc': ['h'], 'diff': []}}


In [129]:
complete = ((1, 1, 1, 1),(1, 0, 1, 1, 0))

In [133]:
for comp in complete:
    for c in comp:
        print(c)

1
1
1
1
1
0
1
1
0


In [138]:
for group in xnat_paths:
    for key in xnat_paths[group]:
        print(len(xnat_paths[group][key]))

1
1
1
1
2
2
3
1
0


In [140]:
for group, comp in zip(xnat_paths, complete):
    print(group,comp)

dix (1, 1, 1, 1)
dif (1, 0, 1, 1, 0)


In [141]:
a = []
for group, comp in zip(xnat_paths, complete):
    for key, c in zip(xnat_paths[group], comp):
        a.append(len(xnat_paths[group][key]) - c)
complete = min(a) > -1
duplicates = max(a) > 0

In [142]:
complete

True

In [143]:
duplicates

True

In [144]:
{'dixon': {'inPhase': ['temp/inPhase(1)'], 'outPhase': ['temp/outPhase(1)'], 'fat': ['temp/fat(1)'], 'water': ['temp/water(1)']}, 'diffusion': {'b50': ['temp/b50(1)'], 'b600': [], 'b900': ['temp/b900(1)', 'temp/b900(2)'], 'adc': ['temp/adc(1)'], 'bvals': []}}

{'dixon': {'inPhase': ['temp/inPhase(1)'],
  'outPhase': ['temp/outPhase(1)'],
  'fat': ['temp/fat(1)'],
  'water': ['temp/water(1)']},
 'diffusion': {'b50': ['temp/b50(1)'],
  'b600': [],
  'b900': ['temp/b900(1)', 'temp/b900(2)'],
  'adc': ['temp/adc(1)'],
  'bvals': []}}

In [145]:
paths = {'dixon': {'inPhase': ['temp/inPhase(1)'], 'outPhase': ['temp/outPhase(1)'], 'fat': ['temp/fat(1)'], 'water': ['temp/water(1)']}, 'diffusion': {'b50': ['temp/b50(1)'], 'b600': [], 'b900': ['temp/b900(1)', 'temp/b900(2)'], 'adc': ['temp/adc(1)'], 'bvals': []}}

In [146]:
len(paths)

2

In [149]:
import os
import pandas as pd
import pydicom

In [151]:
    fields = ['InstanceNumber', 'SliceLocation', 'Rows', 'Columns', 'SequenceName', 'SeriesNumber']
    local_dcms = {}
    for group in paths:
        for key in paths[group]:
            for path in paths[group][key]:
                rows = []
                for dirName, subdirList, fileList in os.walk(path):
                    for filename in fileList:
                        if ".dcm" in filename.lower():
                            dict1 = {'FilePath': filename}
                            dcm = pydicom.dcmread(os.path.join(dirName, filename), stop_before_pixels=True)
                            for field in fields:
                                dict1.update({field: getattr(dcm, field)})
                            rows.append(dict1)
                local_dcms[0].append(pd.DataFrame(rows, columns=fields))
                
                

KeyError: 0

In [153]:
paths.keys()

dict_keys(['dixon', 'diffusion'])

In [155]:
a = [['i','o','w','f'],['5','6','9','a']]

In [156]:
for group in a:
    for elem in group:
        print(elem)

i
o
w
f
5
6
9
a


In [160]:
b = []

In [158]:
b.append([])

In [159]:
b

[[]]

In [162]:
for i, group in enumerate(a):
    b.append([])
    for elem in group:
        b[i].append(elem)
        

In [2]:
import pandas as pd

In [21]:
df = pd.read_csv('example.csv')

In [22]:
df.head()

,InstanceNumber,SliceLocation,Rows,Columns,SequenceName,SeriesNumber
0,201,-979.714127,416,512,*fl3d2,53
1,202,-974.714127,416,512,*fl3d2,53
2,203,-969.714127,416,512,*fl3d2,53
3,204,-964.714127,416,512,*fl3d2,53
4,205,-959.714127,416,512,*fl3d2,53


In [31]:
not df['InstanceNumber'].is_monotonic

True

In [9]:
df.reset_index(drop=True,inplace=True)

In [11]:
df['InstanceNumber'] = df.index + 1

In [12]:
df

,InstanceNumber,SliceLocation,Rows,Columns,SequenceName,SeriesNumber
0,1,-979.714127,416,512,*fl3d2,53
1,2,-974.714127,416,512,*fl3d2,53
2,3,-969.714127,416,512,*fl3d2,53
3,4,-964.714127,416,512,*fl3d2,53
4,5,-959.714127,416,512,*fl3d2,53
5,6,-954.714127,416,512,*fl3d2,53
6,7,-949.714127,416,512,*fl3d2,53
7,8,-944.714127,416,512,*fl3d2,53
8,9,-939.714127,416,512,*fl3d2,53
9,10,-934.714127,416,512,*fl3d2,53


In [18]:
a = 5

In [19]:
a += 1

In [20]:
a

6

In [33]:
dict1 = {'FilePath': 'boop'}

In [34]:
dict1

{'FilePath': 'boop'}

In [35]:
dict1.update({'InstanceNumber': 6})

In [36]:
dict1

{'FilePath': 'boop', 'InstanceNumber': 6}

In [38]:
dict1.update({'FileWhack':'ehe/egaeh/ehe/'})

In [39]:
dict1

{'FilePath': 'boop', 'InstanceNumber': 6, 'FileWhack': 'ehe/egaeh/ehe/'}

In [41]:
pd.DataFrame([dict1])

,FilePath,FileWhack,InstanceNumber
0,boop,ehe/egaeh/ehe/,6


In [42]:
import pydicom

In [43]:
path = 'temp/inPhase(1)/20181215_152525_Avanto/scans/51-T1_IP/resources/DICOM/files/1.2.840.113564.9.1.2843759204.47.2.5001567892-51-1-x7zcbl.dcm'

In [50]:
ds = pydicom.dcmread(path)

In [51]:
ds.InstanceNumber = 15

In [52]:
ds.save_as('temp/reee.dcm')

In [54]:
import pandas as pd
import numpy as np

df = pd.DataFrame([{'c1':10, 'c2':100}, {'c1':11,'c2':110}, {'c1':12,'c2':120}])

In [59]:
for index, row in df.itertuples():
    print(row['c1'], row['c2'])

ValueError: too many values to unpack (expected 2)

In [73]:
df = pd.read_csv('example.csv')

In [74]:
df.columns

Index(['Columns', 'InstanceNumber', 'Rows', 'SequenceName', 'SeriesNumber',
       'SliceLocation', 'path'],
      dtype='object')

In [67]:
def myfunc(path,instance_number):
    ds = pydicom.dcmread(path)
    ds.InstanceNumber = instance_number
    #ds.save_as(path)   
    

In [85]:
df[['InstanceNumber','path']]

,InstanceNumber,path
0,1,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
1,2,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
2,3,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
3,4,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
4,5,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
5,6,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
6,7,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
7,8,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
8,9,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
9,10,temp/inPhase(1)/20181215_152525_Avanto/scans/5...


In [87]:
df[['InstanceNumber','path']].apply(myfunc())

TypeError: myfunc() missing 2 required positional arguments: 'path' and 'instance_number'

https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas/41022840#41022840

https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas/55557758#55557758

https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe